In [ ]:
import pandas as pd
import numpy as np
from urllib.request import urlopen   
from zipfile import ZipFile
import os

: 

In [ ]:
URL = 'https://s3.amazonaws.com/capitalbikeshare-data/2011-capitalbikeshare-tripdata.zip'
url = urlopen(URL)
output = open('zipFile.zip','wb')
output.write(url.read())
output.close()
df = pd.read_csv('zipFile.zip')
os.remove('zipFile.zip')

: 

In [3]:
# extract the month & day of the week for each ride
# & also the minute (over 24 hours i.e. between 0 and 24*60)

tmp = df['Start date'].str.split('-',expand=True)
df['Month'] = tmp[1].astype(int)
tmp = tmp[2].str.split(' ',expand=True)
df['Day'] = tmp[0].astype(int)
tmp = tmp[1].str.split(':',expand=True)
df['Minute'] = 60 * tmp[0].astype(int) + tmp[1].astype(int)
df['Day of week'] = (4 + df['Day'] +\
   np.array([0,31,28,31,30,31,30,31,31,30,31,30]).cumsum()[df.Month-1])%7
df.Duration = np.log(df.Duration)

# public holidays: 
# labor day, columbus day, halloween, veterans day, thanksgiving +/- 1 day
df['Holiday'] = ((df['Month']==9)&(df['Day']==5))| ((df['Month']==10)&(df['Day']==10)) | \
    ((df['Month']==10)&(df['Day']==31))| ((df['Month']==11)&(np.abs(df['Day']-24)<=1))

df = df[(df['Day of week'] <= 4) & (df.Month >= 9) & (df.Month <= 11)  & (df.Holiday == False)]
# keep only regular weekdays in sept & nov (train) or oct (test)


In [4]:
def duration_mean_by_time(group):
    # estimate E[log(Duration)] & Var[log(Duration)] 
    # for each time of day (in minutes), for that particular route
    n = len(group)
    h = 20 # bandwidth in minutes
    K = np.exp(-np.power(12*60 - np.abs(np.abs(group.Minute[:,None] - group.Minute[None,:])\
               - 12*60),2)/2/h**2) * np.array(group.Month != 10)
    # K(i,j) is nonzero only for j in training set
    group['K_weights_total'] = np.dot(K, np.ones(n))
    group['Duration_mean'] = np.divide(np.dot(K, group.Duration), group.K_weights_total, \
            out=np.zeros(n), where= (group.K_weights_total!=0))
    group['Duration_var'] = np.divide(np.dot(K, np.power(group.Duration,2)), group.K_weights_total, \
            out=np.zeros(n), where= (group.K_weights_total!=0)) - np.power(group.Duration_mean,2)
    return group

In [5]:
df = df.groupby(['Start station number', 'End station number']).apply(duration_mean_by_time)
df1 = df[(df.Month == 10) & (df.K_weights_total >= 20)] # test set

In [6]:
# generate CPT copies of X when the conditional distribution is Gaussian
# i.e. X | Z=Z_i ~ N(mu[i],sig2[i])
def generate_X_CPT_gaussian(nstep,M,X0,mu,sig2):
    log_lik_mat = - np.power(X0,2)[:,None] * (1/2/sig2)[None,:] + X0[:,None] * (mu/sig2)[None,:]
    Pi_mat = generate_X_CPT(nstep,M,log_lik_mat)
    return X0[Pi_mat]

# generate CPT copies of X in general case
# log_lik_mat[i,j] = q(X[i]|Z[j]) where q(x|z) is the conditional density for X|Z
def generate_X_CPT(nstep,M,log_lik_mat,Pi_init=[]):
    n = log_lik_mat.shape[0]
    if len(Pi_init)==0:
        Pi_init = np.arange(n,dtype=int)
    Pi_ = generate_X_CPT_MC(nstep,log_lik_mat,Pi_init)
    Pi_mat = np.zeros((M,n),dtype=int)
    for m in range(M):
        Pi_mat[m] = generate_X_CPT_MC(nstep,log_lik_mat,Pi_)
    return Pi_mat

def generate_X_CPT_MC(nstep,log_lik_mat,Pi):
    n = len(Pi)
    npair = np.floor(n/2).astype(int)
    for istep in range(nstep):
        perm = np.random.choice(n,n,replace=False)
        inds_i = perm[0:npair]
        inds_j = perm[npair:(2*npair)]
        # for each k=1,...,npair, decide whether to swap Pi[inds_i[k]] with Pi[inds_j[k]]
        log_odds = log_lik_mat[Pi[inds_i],inds_j] + log_lik_mat[Pi[inds_j],inds_i] \
            - log_lik_mat[Pi[inds_i],inds_i] - log_lik_mat[Pi[inds_j],inds_j]
        swaps = np.random.binomial(1,1/(1+np.exp(-np.maximum(-500,log_odds))))
        Pi[inds_i], Pi[inds_j] = Pi[inds_i] + swaps*(Pi[inds_j]-Pi[inds_i]), Pi[inds_j] - \
            swaps*(Pi[inds_j]-Pi[inds_i])   
    return Pi


In [7]:
# create CRT & CPT copies

ntrial = 10
M = 1000 # number of copies
n = len(df1)
nstep = 50 # number of steps in the MC for the CPT

print('Total number training data points: n_train = '+ str((df.Month!=10).sum()))
print('Total number test data points (after screening): n = '+str(n))

X = np.array(df1.Duration)
X_CRT = np.zeros((ntrial,M,n))
X_CPT = np.zeros((ntrial,M,n))

Duration_mean = np.array(df1.Duration_mean)
Duration_var = np.maximum(1e-12,np.array(df1.Duration_var))

for itrial in range(ntrial):
    np.random.seed(12345 + itrial)
    X_CRT[itrial] = np.random.normal(size=(M,n)) * np.sqrt(Duration_var) + Duration_mean
    X_CPT[itrial] = generate_X_CPT_gaussian(nstep,M,X,Duration_mean,Duration_var)

Total number training data points: n_train = 149912
Total number test data points (after screening): n = 7346


In [8]:
def corr_mat_vec(x,y):
    # x is a ntrial-by-M-by-n numpy array, y is a length-n numpy array
    # return ntrial-by-M matrix of correlations of each x with y
    x = ((x.transpose((2,0,1)) - x.mean(2))/np.sqrt(x.var(2))).transpose((1,2,0))
    y = (y - y.mean(0))/np.sqrt(y.var(0))
    return np.dot(x,y)/len(y)

def corr_mat_vec_cat(x,y):
    # x is a ntrial-by-M-by-n numpy array, y is a length-n numpy array
    # y is a categorical variable with c many categories
    # return ntrial-by-M-by-c array of correlations 
    # between each x and each y indicator
    yvals = np.unique(y)
    tmp = np.zeros((x.shape[0],x.shape[1],len(yvals)))
    for i in range(len(yvals)):
        tmp[:,:,i] = corr_mat_vec(x,(y==yvals[i]))
    return tmp

def str_mean_se(vec):
    return str(np.round(vec.mean(),4)) + ' (' + str(np.round(np.sqrt(vec.var()/len(vec)),4)) + ')'

In [9]:
# Response 1: member (1) or nonmember i.e. 'casual' (0)
Y1 = np.array(df1['Member type'] == 'Member')
T = np.abs(corr_mat_vec((X - Duration_mean).reshape((1,1,-1)),Y1))
T_CRT = np.abs(corr_mat_vec((X_CRT - Duration_mean),Y1))
T_CPT = np.abs(corr_mat_vec((X_CPT - Duration_mean),Y1))
p_CRT = (1 + (T_CRT>=T).sum(1))/(1+M)
p_CPT = (1 + (T_CPT>=T).sum(1))/(1+M)

print('Response Y1 = member type: p-value = ' +\
      str_mean_se(p_CPT) + ' for CPT, ' + str_mean_se(p_CRT) + ' for CRT')

Response Y1 = member type: p-value = 0.001 (0.0) for CPT, 0.001 (0.0) for CRT


In [10]:
# Response 2: date
Y2 = np.array(df1['Day'])
T = np.abs(corr_mat_vec((X - Duration_mean).reshape((1,1,-1)),Y2))
T_CRT = np.abs(corr_mat_vec((X_CRT - Duration_mean),Y2))
T_CPT = np.abs(corr_mat_vec((X_CPT - Duration_mean),Y2))
p_CRT = (1 + (T_CRT>=T).sum(1))/(1+M)
p_CPT = (1 + (T_CPT>=T).sum(1))/(1+M)

print('Response Y2 = date: p-value = ' +\
      str_mean_se(p_CPT) + ' for CPT, ' + str_mean_se(p_CRT) + ' for CRT')

Response Y2 = date: p-value = 0.1146 (0.0032) for CPT, 0.1293 (0.0032) for CRT


In [11]:
# Response 3: day of week
Y3 = np.array(df1['Day of week'])
T = np.abs(corr_mat_vec_cat((X-Duration_mean).reshape((1,1,-1)),Y3)).max(2)
T_CRT = np.abs(corr_mat_vec_cat((X_CRT - Duration_mean),Y3)).max(2)
T_CPT = np.abs(corr_mat_vec_cat((X_CPT - Duration_mean),Y3)).max(2)
p_CRT = (1 + (T_CRT>=T).sum(1))/(1+M)
p_CPT = (1 + (T_CPT>=T).sum(1))/(1+M)

print('Response Y3 = day of week: p-value = ' +\
      str_mean_se(p_CPT) + ' for CPT, ' + str_mean_se(p_CRT) + ' for CRT')

Response Y3 = day of week: p-value = 0.198 (0.0037) for CPT, 0.2063 (0.0032) for CRT
